In [3]:
import fiftyone as fo
from ultralytics import YOLO

def load_dataset(split):
    """
    Load a CarDD dataset split from COCO format files.

    Args:
        split (str): Dataset split to load. Must be one of: 'train', 'test', or 'val'.

    Returns:
        fiftyone.core.dataset.Dataset: A FiftyOne dataset containing the car damage detection
            data for the specified split, with the following fields:
            - Ground truth instance segmentation masks and bounding boxes
            - Original COCO category IDs preserved
            - Named as 'car_dd_{split}'
    """
    dataset = fo.Dataset.from_dir(
        data_path=f"CarDD_release/CarDD_COCO/{split}2017",
        labels_path=f"CarDD_release/CarDD_COCO/annotations/instances_{split}2017.json",
        dataset_type=fo.types.COCODetectionDataset,
        name=f"car_dd_{split}",
        overwrite=True,
        include_id=True,
    )
    return dataset


In [4]:
train_data = load_dataset("train")
test_data = load_dataset("test")
val_data = load_dataset("val")

 100% |███████████████| 2816/2816 [15.9s elapsed, 0s remaining, 182.2 samples/s]      
 100% |█████████████████| 374/374 [2.0s elapsed, 0s remaining, 194.9 samples/s]      
 100% |█████████████████| 810/810 [4.4s elapsed, 0s remaining, 186.8 samples/s]      


FiftyOne supports exporting datasets to disk in various common formats and can be extended for custom formats.

 You can export entire datasets or subsets using the Python library or CLI. The export() method automatically coerces data types to match the requested export types. Several dataset types are supported, including ImageDirectory, VideoDirectory, FiftyOneImageClassificationDataset, and COCODetectionDataset. You can also export datasets in custom formats by defining your own Dataset or DatasetExporter class.


https://beta-docs.voxel51.com/fiftyone_concepts/export_datasets/

In [ ]:


def export_to_yolo(dataset, split, classes = ["crack", "dent", "scratch", "glass shatter", "tire flat", "lamp broken"]):
    export_path = f"/Users/prerna/datasets/carDD"
    label_field = "polylines"
    # fol.instances_to_polylines(dataset, "segmentations", "polylines")
    dataset.export(
        export_dir=export_path,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        label_field=label_field,
        classes=classes,
    )


def train():
    # Load YOLOv11 model for instance segmentation
    model = YOLO('yolo11s-seg.pt')  # Pre-trained YOLOv11 segmentation model

    # Set up training configuration
    model.train(
        data='/Users/prerna/datasets/carDD/dataset.yaml',  
        epochs=100,                                        
        batch=8,                                          
        imgsz=640,                                         
        val=True,                                          
        save=True,                                         
        save_period=5,                                     
        project='car_damage_segmentation',                 
        name='yolov11_seg_run',                            
        workers=4,                                         
        device='mps',                                      
        patience=10,                                       
        verbose=True                                       
    )

def apply_model(dataset):
    model_path = "/Users/prerna/models/carDD/best.pt"
    model = YOLO(model_path)

    dataset = fo.load_dataset("carDD-test")
    dataset.apply_model(model, label_field="yolo11_large")